In [1]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import random
import zipfile
device='cpu'

In [2]:
def load_data_jay_lyrics():
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars=f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([( char,i ) for i , char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [3]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [5]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)
rnn_layer2=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens,num_layers=2)
rnn_layer

RNN(1027, 256)

In [6]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
Y,state_new=rnn_layer(X,state)
print(X.shape)
print(Y.shape,len(state_new),state_new[-1].shape)

torch.Size([35, 2, 1027])
torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [7]:
def to_onehot(x,n_class):
    def one_hot(x,n_class):
        x=x.long()
        res=torch.zeros(x.shape[0],n_class)
        res=res.scatter(1,x.view(-1,1),1)
        return res
    return [one_hot(x[:,i],n_class) for i in range(x.shape[1])]


In [8]:
x=torch.arange(10).view(2,5)
to_onehot(x,10)

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])]

In [10]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super().__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,self.vocab_size)
        self.state=None
        
    def forward(self,inputs,state):
        X=to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state

In [13]:
torch.stack(to_onehot(x,10)).shape

torch.Size([5, 2, 10])

In [14]:
def predict_rnn_pytorch(perfix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state=None
    outputs=[char_to_idx[perfix[0]]]
    for t in range(num_chars+len(perfix)-1):
        X=torch.Tensor([outputs[-1]]).view(1,1)
        if state is not None:
            if isinstance(state,tuple):
                state=(state[0],state[1])
            else:
                state=state
        (Y,state) = model(X,state)
        if t<len(perfix)-1:
            outputs.append(char_to_idx[perfix[t+1]])
        else:
            outputs.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in outputs])

In [15]:
model=RNNModel(rnn_layer,vocab_size)
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开友星非墟驳凉绕进北北'

In [16]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.Tensor(corpus_indices,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [17]:
def grad_clipping(params,theta,device):
    norm=torch.Tensor([0.0])
    for p in params:
        norm+=(p.grad.data **2).sum()
    norm=norm.sqrt().item()
    if norm > theta:
        for p in params:
            p.grad.data *= (theta/norm)

In [24]:
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,num_epochs,num_steps,lr,
                                 clipping_theta,batch_size,pred_period,pred_len,prefixes):
    loss=nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_consecutive(corpus_indices,batch_size,num_steps)
        for x,y in data_iter:
            if state is not None:
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
            (output,state)=model(x,state)
            y=torch.transpose(y,0,1).contiguous().view(-1)
            l=loss(output,y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(),clipping_theta,device)
            optimizer.step()
            
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
            
        try:
            perplexity=math.exp(l_sum/n)
        except OverflowError:
            perplexity=float('inf')
        if (epoch+1) % pred_period ==0:
            print('epoch %d,perplexity %f,time %.2f sec' % (epoch+1,perplexity,time.time()-start))
            
            for perfix in prefixes:
                print('-',predict_rnn_pytorch(perfix,pred_len,model,vocab_size,device,idx_to_char,char_to_idx))
            
                

In [25]:
num_steps=35
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                              corpus_indices, idx_to_char, char_to_idx,
                              num_epochs, num_steps, lr, clipping_theta,
                              batch_size, pred_period, pred_len, prefixes)

epoch 50,perplexity 7.623246,time 0.93 sec
- 分开始不过 语沉默 说你我 说你开 我不多 爱 我不要再想你 不知我觉 你想要你的你 我有了这节奏 后知
- 不分开 我想你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
epoch 100,perplexity 1.249788,time 0.96 sec
- 分开 我的大声宣 用跟你 我不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知
- 不分开 你在人海起 决家就在发现迷了知道你已一场悲剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的回忆 
epoch 150,perplexity 1.063314,time 0.85 sec
- 分开 我的大声宣  对你 回不知 这样没有妈离 我和妈看棒球 想你 想要你 陪着去 别怪我  你 你知后
- 不分开 你在我胸起  想就这样 抛物线进球  手过了运球 篮下 传出就让我遇见你是你场悲剧 我想你看棒球 
epoch 200,perplexity 1.033987,time 0.93 sec
- 分开 我的大界休 啦啦  说穿了其实我的愿望就怎么小 就怎么每天祈祷我的心跳你知道  杵在伊斯坦堡 却只
- 不分开 已经离透开 不知不想 你是黑色的泪不能 语沉默 娘子却依旧每日折一枝杨柳 在小村外的溪边河口 默默
epoch 250,perplexity 1.020488,time 0.85 sec
- 分开 我的大界休 啦我 你和文 有词再 你 靠有我不要再也 我说 你不 我说你 是你说没 你不知 我已经
- 不分开 你在练感动 没有一口让我知道的太从小到 你打我手 你不要 没有的爱 你 后悔我 你脸风 在人出人 
